In [ ]:
import sys
sys.path.append("/Users/fabio/jars")
import pandas as pd
import numpy as np
import tqdm
from pyspark.sql.window import Window
from sklearn.preprocessing import LabelEncoder
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark import SparkContext
from graphframes import GraphFrame
from pyspark.sql.types import *
import multiprocessing
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
import os
import shutil
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from collections import Counter
import math

In [ ]:
spark_driver_memory = "10g"
spark_executor_memory = "6g"
spark_partial_results_folder = './partial_results'


spark = SparkSession.builder \
                    .config("spark.driver.memory", spark_driver_memory) \
                    .config("spark.executor.memory", spark_executor_memory) \
                    .master("local[*]") \
                    .config("spark.sql.autoBroadcastJoinThreshold", 100 * 1024 * 1024)\
                    .config("spark.driver.port", 4040) \
                    .config("spark.driver.bindAddress", "127.0.0.1") \
                    .getOrCreate()
print("Spark session created")
sc = spark.sparkContext
print("Spark context created")


if not os.path.exists(spark_partial_results_folder):
    os.makedirs(spark_partial_results_folder)

## Load data

In [ ]:
# Definizione dello schema per la lettura del file 
schema = StructType([
    StructField('timestamp', StringType(), True),
    StructField('from_bank', IntegerType(), True),
    StructField('from_account', StringType(), True),
    StructField('to_bank', IntegerType(), True),
    StructField('to_account', StringType(), True),
    StructField('amount_received', FloatType(), True),
    StructField('receiving_currency', StringType(), True),
    StructField('amount_paid', FloatType(), True),
    StructField('payment_currency', StringType(), True),
    StructField('payment_format', StringType(), True),
    StructField('is_laundering', IntegerType(), True)])

#Lettura del file csv direttamente in spark
df = spark.read.csv("../dataset/HI-Small_Trans.csv", header = False, schema=schema)

#Rimozione della prima riga in quanto è presente l'header del csv
df = df.filter(col('timestamp') != "Timestamp")

#Aggiunta dell'id univoco per le righe.
df = df.withColumn("index", monotonically_increasing_id())

df.show(5, truncate=False)
df.printSchema()


L'analisi sui dati verrà suddivisa in più parti: 
Inizialmente verrà fatta un'analisi indipendente dall'ordine temporale, che si concentrerà principalmente sul capire la struttura del dataset, le proporzioni tra laundering e non laundering e se ci sono marcate differenze tra le features delle due classi. 
Successivamente si andrà ad esplorare meglio la relazione temporale tra i dati essendo il tempo una delle feature principali quando si cerca di scovare pattern fraudolenti.
Infine, calcolate le features che si pensa possano essere di interesse, verrà calcolata la matrice di correlazione per capire quanto siano correlate le features all'interno del dataset per capirne anche l'importanza ai fini dell'apprendimento del modello.

## Prime analisi sul dataset

In questa sezione andremo a vedere come sono distribuite le classi all'interno del dataset, andando a studiare anche la proporzione di transazioni fraudolente per le diverse tipologie di metodi di pagamento e di formato, oltre alla differenza, se presente, tra amount che vengono spostati quando le transazioni sono fraudolente.

In [ ]:
# Controllo se ci sono valori nulli in qualche cella
df.select([F.sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

Non sono presenti valori nulli, quindi non è necessario rimuovere o modificare alcuna riga. Dunque si può procedere con le successive analisi

In [ ]:
#Proporzione tra laundering e non laundering all'interno del dataset. 1: Laundering, 0: Not laundering
total_count = df.count()
df.select('is_laundering').groupBy('is_laundering').agg(count('*').alias('count')).withColumn("proportion", col('count')/total_count).show(5, truncate=False)

Fin da subito è possibile osservare che le transazioni fraudolente sono all'incirca 1/1000 di quelle totali. Questo porta ad avere un dataset altamente sbilanciato e rende l'identificazione di pattern più complessa.

### Display payment format in relation to laundering transaction

In [ ]:
df.select('payment_format', 'is_laundering') \
    .groupBy('payment_format') \
    .agg(
        sum(col('is_laundering').cast('int')).alias('1'),
        sum((1 - col('is_laundering')).cast('int')).alias('0')
    ) \
    .withColumn("proportion", (col("1") / col("0")).cast('Decimal(20,6)')) \
    .orderBy('proportion', ascending=False) \
    .show(truncate=False)

# Calculate the number of corresponding values for each value of the "Payment Format" and "Is Laundering" columns
grouped_df = df.groupBy("payment_format", "is_laundering").count()

count_values = grouped_df.toPandas()
count_values_payment = count_values.pivot(index='payment_format', columns='is_laundering', values='count')


fig, axs = plt.subplots(1, 2, figsize=(15, 6))
bar_width = 0.35
bar_positions = range(len(count_values_payment.index))

#Arithmetic scale
axs[0].bar(bar_positions, count_values_payment[0], bar_width, label='Is Laundering = 0')
axs[0].bar([p + bar_width for p in bar_positions], count_values_payment[1], bar_width, label='Is Laundering = 1')
axs[0].set_xticks(bar_positions)
axs[0].set_xticklabels(count_values_payment.index, rotation='vertical') 
axs[0].set_xlabel('Payment Format')
axs[0].set_ylabel('Number of corresponding values')
axs[0].set_title('Bar chart in arithmetic scale')
axs[0].legend()

#Logaritmic scale
axs[1].bar(bar_positions, count_values_payment[0], bar_width, label='Is Laundering = 0')
axs[1].bar([p + bar_width for p in bar_positions], count_values_payment[1], bar_width, label='Is Laundering = 1')
axs[1].set_xticks(bar_positions)
axs[1].set_xticklabels(count_values_payment.index, rotation='vertical') 
axs[1].set_xlabel('Payment Format')
axs[1].set_ylabel('Number of corresponding values')
axs[1].set_title('Bar chart in logarithmic scale')
axs[1].legend()
axs[1].set_yscale('log')

# Show the chart
plt.show()

L'analisi sul formato di pagamento in relazione alle transazioni laundering mostra in maniera abbastanza evidente che la metodologia di pagamento ACH è molto spesso correlata ad una transazione fraudolenta, mentre, ancor più interessante, le metodologie Reinvestment e Wire non contengono alcun tipo di transazione fraudolenta. Questo fa intuire fin da subito che il formato di pagamento è una feature essenziale da tenere in considerazione, in quanto già da sola, se usata con un modello ad albero, dovrebbe essere in grado di classificare correttamente tutte le transazioni Reinvestment e Wire.

### Display payment currency in relation to laundering transaction

In [ ]:
df.select('payment_currency', 'is_laundering') \
    .groupBy('payment_currency') \
    .agg(
        sum(col('is_laundering').cast('int')).alias('1'),
        sum((1 - col('is_laundering')).cast('int')).alias('0')
    ) \
    .withColumn("proportion", (col("1") / col("0")).cast('Decimal(20,6)')) \
    .orderBy('proportion', ascending=False) \
    .show(truncate=False)

grouped_df = df.groupBy("payment_currency", "is_laundering").count()

# Convert Spark DataFrame to Pandas DataFrame
count_values = grouped_df.toPandas()

# Use the unstack() method
count_values_currency = count_values.pivot(index='payment_currency', columns='is_laundering', values='count')

# Sort the values by Is Laundering = 1 in descending order
count_values_currency = count_values_currency.sort_values(1, ascending=False)

# Create a bar chart with a logarithmic scale
fig, axs = plt.subplots(1, 2, figsize=(15, 6))
bar_width = 0.35
bar_positions = range(len(count_values_currency.index))
axs[0].bar(bar_positions, count_values_currency[0], bar_width, label='Is Laundering = 0')
axs[0].bar([p + bar_width for p in bar_positions], count_values_currency[1], bar_width, label='Is Laundering = 1')
axs[0].set_xticks(bar_positions)
axs[0].set_xticklabels(count_values_currency.index, rotation='vertical') 
axs[0].set_xticklabels(count_values_currency.index)
axs[0].set_xlabel('Payment Currency')
axs[0].set_ylabel('Number of corresponding values')
axs[0].set_title('Bar chart in arithmetic scale')
axs[0].legend()

axs[1].bar(bar_positions, count_values_currency[0], bar_width, label='Is Laundering = 0')
axs[1].bar([p + bar_width for p in bar_positions], count_values_currency[1], bar_width, label='Is Laundering = 1')
axs[1].set_xticks(bar_positions)
axs[1].set_xticklabels(count_values_currency.index, rotation='vertical') 
axs[1].set_xticklabels(count_values_currency.index)
axs[1].set_xlabel('Payment Currency')
axs[1].set_ylabel('Number of corresponding values')
axs[1].set_title('Bar chart in logarithmic scale')
axs[1].legend()
axs[1].set_yscale('log')

# Show the chart
plt.show()

Lo studio sulle metodologie di pagamento in relazione alle transazioni fraudolente risulta essere meno incisivo di quello sul formato di pagamento. In ogni caso dimostra come la tipologia di pagamento US Dollar, che sembra essere anche la più frequente all'interno del dataset, contiene il maggior numero di transazioni fraudolente. Nonostante questo, a livello di proporzione, sembra essere molto più fraudolento il metodo di pagamento Saudi Riyal, seguito da Euro.

### Display relationhip between amount paid and laundering transaction

In [ ]:
grouped_stats = df.groupBy('is_laundering').agg(
    min(col('amount_paid')).alias('min').cast('Decimal(20,6)').alias('min'),
    max(col('amount_paid')).alias('max').alias('max'),
    mean(col('amount_paid')).alias('mean').alias('mean')
)

grouped_stats.show()

# Sottocampiona la classe maggioritaria
df_0 = df.filter(df['is_laundering'] == 0).sample(withReplacement=False, fraction=100000/5000000)
df_1 = df.filter(df['is_laundering'] == 1)
df_sampled = df_0.union(df_1)


df_pd = df_sampled.toPandas()

plt.yscale("log")
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: format(x, ',.2f')))

# Plotta i punti per is_laundering == 0 in blu e is_laundering == 1 in rosso
plt.scatter(df_pd[df_pd['is_laundering'] == 0]['is_laundering'], df_pd[df_pd['is_laundering'] == 0]['amount_paid'], color='blue', alpha=0.5, label='Not Laundering')
plt.scatter(df_pd[df_pd['is_laundering'] == 1]['is_laundering'], df_pd[df_pd['is_laundering'] == 1]['amount_paid'], color='red', alpha=0.5, label='Laundering')

plt.title("Relationship between Is Laundering and Amount Paid")
plt.xlabel("Is Laundering")
plt.ylabel("Amount Paid")
plt.xticks([0, 1])
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


A differenza delle precedenti analisi, andando a studiare il comportamento che hanno le transazioni fraudolente rispetto all'ammontare della somma inviata, si nota come non ci sia alcun tipo di separazione tra transazione fraudolenta o non fraudolenta. Infatti, utilizzando un grafico, è possibile notare come il range di amount delle transazioni fraudolenti è contenuto all'interno di quelle non fraudolenti senza un effettivo stacco. Oltretutto, considerando che le transazioni fraudolenti sono circa 1/1000 di quelle non fraudolenti, si può evincere che molto probabilmente l'amount della transazione sporca e non aiuta la capacità del modello a classificare correttamente i dati. 

Le due features verranno comunque mantenute per osservarle meglio nella matrice di correlazione.

### Display top 10 accounts for fraudolent transactions

In [ ]:
# Calcola il numero totale di transazioni per ogni account
total_transactions = df.groupBy('from_account')\
    .agg(F.count('*').alias('total_trans'))

# Conta il numero di transazioni fraudolente per ogni account
fraudulent_transactions = df.filter(col('is_laundering') == 1)\
    .groupBy('from_account')\
    .agg(F.count('*').alias('count_laundering'))

# Unisci i due conteggi
joined_df = total_transactions.join(fraudulent_transactions, 'from_account', 'left_outer')\
    .fillna(0)  # In caso non ci siano transazioni fraudolente per un determinato account

# Calcola il rapporto
joined_df = joined_df.withColumn('fraud_rate', (F.col('count_laundering') / F.col('total_trans')).cast('Decimal(20,6)'))

# Ordina e mostra i risultati
joined_df.orderBy('total_trans', ascending=False).show(10)
print(f"Number of account that send just one transaction and it is fraudolent: {joined_df.filter((col('total_trans') == 1) & (col('fraud_rate') == 1)).count()}")

### Find how many times an account send laundering and not laundering transaction to same account

In [ ]:
df_temp = df.select('from_account', 'to_account', 'is_laundering')

# Raggruppa e conta le occorrenze uniche
grouped_df = df_temp.groupBy('from_account', 'to_account').agg(collect_set('is_laundering').alias('unique_values'))

# Filtra i risultati con più di una occorrenza
filtered_df = grouped_df.filter(col('unique_values').getItem(0) != col('unique_values').getItem(1))

# Calcola il numero di occorrenze filtrate per ogni 'from_account'
result_df = filtered_df.groupBy('from_account').count().orderBy(col('count').desc())

# Mostra i primi 10 risultati
result_df.show(10)

Per quanto riguarda gli account si evince come ci siano account che hanno effettuato nettamente più transazioni di altri. A parte questo, l'utilizzo degli accoun come features non è rilevante ai fini dell'allenamento del modello. Possiamo però andare a studiare il comportamento delle transazioni quando gli account sono uguali o diversi e capire se c'è una particolare correlazione con transazioni fraudolente. Stesso può essere fatto con la valuta di partenza e di arrivo e con l'amount di partenza e di arrivo.

### Show relationship between same account and fraudolent transaction

In [ ]:
# Filtra le righe e raggruppa per is_laundering
grouped_df = df.filter(F.col('from_account') == F.col('to_account'))\
    .groupBy('is_laundering')\
    .agg(F.count('*').alias('count'))

count_total = df.groupBy(col('is_laundering').alias('is_laundering_temp')).agg(count('*').alias('count_total'))
grouped_df = grouped_df.join(count_total, col('is_laundering') == col('is_laundering_temp')).drop('is_laundering_temp').withColumn('proportion', (col('count') / col('count_total')).cast('Decimal(20,6)')).drop('count_total')
grouped_df.show()

Da queste analisi risulta molto poco probabile che una transazione che contenga lo stesso account di partenza e di destinazione sia fraudolenta. Questa potrebbe rilevarsi una features interessante da considerare e aggiungere.

In [ ]:
def add_same_account(df):
    return df.withColumn('same_account', when((col('from_account') == col('to_account')), 1).otherwise(0))

In [ ]:
# Aggiunta della feature che considera se l'account di partenza e destinazione sono uguali
df = add_same_account(df)

### Show relationship between same bank and fraudolent transaction

In [ ]:
# Filtra le righe e raggruppa per is_laundering
grouped_df = df.filter(F.col('from_bank') == F.col('to_bank'))\
    .groupBy('is_laundering')\
    .agg(F.count('*').alias('count'))

count_total = df.groupBy(col('is_laundering').alias('is_laundering_temp')).agg(count('*').alias('count_total'))
grouped_df = grouped_df.join(count_total, col('is_laundering') == col('is_laundering_temp')).drop('is_laundering_temp').withColumn('proportion', (col('count') / col('count_total')).cast('Decimal(20,6)')).drop('count_total')
grouped_df.show()

In [ ]:
from_df = df.select(col("from_account").alias("account"), col("from_bank").alias("bank"))
to_df = df.select(col("to_account").alias("account"), col("to_bank").alias("bank"))
combined_df = from_df.union(to_df)

# Raggruppa per account e conta le banche distinte
result_df = combined_df.groupBy("account").agg(countDistinct("bank").alias("num_banks"))

# Filtra gli account con più di una banca associata
filtered_df = result_df.filter(result_df["num_banks"] > 1)

show_df = filtered_df.orderBy('num_banks', ascending=False)
print(f'Number of account with more than one bank associated: {show_df.count()}')
# 4. Mostra i risultati
show_df.show()

Lo studio dimostra che ci sono 8 account che hanno più di una banca associata e che il massimo di banche associate è 2. Inoltre dimostra che c'è una più forte correlazione tra la banca di invio e ricezione uguale rispetto allo stesso account. Ovviamente una stessa banca può appartenere a più utenti. E' anche questa una feature che potrebbe tornare utile nella classificazione.

In [ ]:
def add_same_bank(df):
    return df.withColumn('same_bank', when((col('from_bank') == col('to_bank')), 1).otherwise(0))

In [ ]:
# Aggiunta della feature che considera se l'account di partenza e destinazione sono uguali
df = add_same_bank(df)

### Show relationship between same amount and fraudolent transaction

In [ ]:
# Filtra le righe e raggruppa per is_laundering
grouped_df = df.filter(F.col('amount_received') == F.col('amount_paid'))\
    .groupBy('is_laundering')\
    .agg(F.count('*').alias('count'))

count_total = df.groupBy(col('is_laundering').alias('is_laundering_temp')).agg(count('*').alias('count_total'))
grouped_df = grouped_df.join(count_total, col('is_laundering') == col('is_laundering_temp')).drop('is_laundering_temp').withColumn('proportion', (col('count') / col('count_total')).cast('Decimal(20,6)')).drop('count_total')
grouped_df.show()

In questo caso possiamo vedere che tutte le transazioni che sono fraudolente hanno l'amount inviato e ricevuto uguale. Di conseguenza possiamo dedurre che abbiano anche la stessa valuta di invio e ricezione, in quanto sennò ci sarebbe una differenza dovuta al cambio valuta. In ogni caso, seppur questa possa sembrare una feature interessante, mettendola in relazione anche con le transazioni non fraudolente, risulta che non c'è molta differenza. In poche parole, qualora amount received e send non fossero uguali, la classificazione sarebbe corretta. C'è però da considerare che solo l'1,5% del dataset presenta questa caratteristica, dunque potrebbe non essere così impattante. 

Effettuiamo comunque un'analisi per vedere anche se ci sono casi in cui l'amount è diverso nonostante le currencies siano uguali

### Show relationship between same currency and fraudolent transaction

In [ ]:
# Filtra le righe e raggruppa per is_laundering
grouped_df = df.filter(F.col('receiving_currency') == F.col('payment_currency'))\
    .groupBy('is_laundering')\
    .agg(F.count('*').alias('count'))

count_total = df.groupBy(col('is_laundering').alias('is_laundering_temp')).agg(count('*').alias('count_total'))
grouped_df = grouped_df.join(count_total, col('is_laundering') == col('is_laundering_temp')).drop('is_laundering_temp').withColumn('proportion', (col('count') / col('count_total')).cast('Decimal(20,6)')).drop('count_total')
grouped_df.show()

Questa analisi dimostra che non c'è una correlazione 1:1 tra amount e currency in quanto si evince che ci siano transazioni che non hanno la stessa valuta di invio e ricezione ma hanno lo stesso amount. La differenza è però trascurabile. In ogni caso, considerando che anche questo studio ha portato a dimostrare che tutte le transazioni fraudolente hanno la stessa valuta, ma meno transazioni non fraudolente hanno la stessa valuta, andiamo a considerare solo 'same_amount' come feature da aggiungere in quanto, seppur di poco, ci offre uno stacco maggiore di proporzione rispetto ai laundering.

Rimane comunque che questa feature non porta ad una separazione delle classi considerevole e dunque in un processo di feature selection potrebbe venir scartata.

In [ ]:
def add_same_currency(df):
    return df.withColumn('same_currency', when((col('receiving_currency') == col('payment_currency')), 1).otherwise(0))

In [ ]:
# Aggiunta della feature che considera se la valuta di partenza e destinazione sono uguali
df = add_same_currency(df)

## Analisi considerando il timestamp

Come prima cosa è necessario andare a fare un casting del timestamp da stringa a DateType. Oltre a questo vado a separare il timestamp in componenti separate così da poterlo analizzare più facilmente.

In [ ]:
df = df.withColumn("timestamp", to_timestamp("timestamp", "yyyy/MM/dd HH:mm"))

# Split the timestamp column into separate components
df = df.withColumn("year", year("timestamp"))\
                             .withColumn("month", month("timestamp"))\
                             .withColumn("day", dayofmonth("timestamp"))\
                             .withColumn("hour", hour("timestamp"))\
                             .withColumn("minute", minute("timestamp"))


# Dato che ho fatto delle aggiunte di features, vado a fare il caching del dataset per migliorare le performance                
df.cache()
df.show(5)

### Show laundering per componenti del timestamp

In [ ]:
def laundering_for(df, col_name: str):
    print(f"Laundering for {col_name}")
    df.select(col_name, 'is_laundering') \
    .groupBy(col_name) \
    .agg(
        sum(col('is_laundering').cast('int')).alias('count(1)'),
        sum((1 - col('is_laundering')).cast('int')).alias('count(0)'),
    ).withColumn("ratio", (col('count(1)')/col('count(0)')).cast('Decimal(20,6)')) \
  .orderBy(col('ratio').desc()) \
  .show(truncate=False)

#### Laundering per Anno

In [ ]:
laundering_for(df, 'year')

# Remove year feature
df = df.drop('year')

Il dataset presenta un solo anno di transazioni, quindi come feature 'year' risulta inutile. Per questo motivo andiamo a rimuoverla.

#### Laundering per Mese

In [ ]:
laundering_for(df, 'month')

# Remove year feature
df = df.drop('month')

Come per l'anno, anche 'month' ha un solo valore per tutte le righe. Procediamo dunque a rimuoverla.

#### Laundering per Giorno

In [ ]:
laundering_for(df, 'day')

#### Laundering per hour

In [ ]:
laundering_for(df, 'hour')

#### Laundering for Week

In [ ]:
laundering_for(df, 'minute')

A differenza del mese e dell'anno, i giorni, le ore e i minuti possono essere considerati come feature importanti. Si nota infatti come determinati valori sono più propensi ad essere fraudolenti rispetto ad altri. Andremo ad utilizzare queste tre nuove features per calcolarne di nuove in relazione agli account. 

### Count the number of transactions an account receive in different period of time

In [ ]:
def add_trans_received(df):
    window = Window.partitionBy('to_account', 'day')
    df = df.withColumn("transaction_received_per_day", count('*').over(window))

    window = Window.partitionBy('to_account', 'hour')
    df = df.withColumn("transaction_received_per_hour", count('*').over(window))

    window = Window.partitionBy('to_account', 'week')
    df = df.withColumn("transaction_received_per_week", count('*').over(window))

    return df

In [ ]:
df = add_trans_received(df)

### Count the number of transactions an account send in different period of time

In [ ]:
def add_trans_send(df):
    window = Window.partitionBy('from_account', 'day')
    df = df.withColumn("transaction_send_per_day", count('*').over(window))

    window = Window.partitionBy('from_account', 'hour')
    df = df.withColumn("transaction_send_per_hour", count('*').over(window))

    window = Window.partitionBy('from_account', 'week')
    df = df.withColumn("transaction_send_per_week", count('*').over(window))

    return df

In [ ]:
df = add_trans_send(df)

### Count the minutes since last transaction send by an Account

In [ ]:
def add_minutes_since_last_trans(df):

    windowSpec = Window.partitionBy("from_account").orderBy("timestamp")

    # Usa la funzione 'lag' per ottenere il timestamp della transazione precedente
    df = df.withColumn("prev_timestamp", F.lag(df.timestamp).over(windowSpec))

    # Calcola la differenza in minuti, -1 se è la prima transazione che effettua
    df = df.withColumn(
        "minutes_since_last_transaction", 
        F.when(
            F.isnull(df.prev_timestamp), 
            -1
        ).otherwise(
            (F.unix_timestamp(df.timestamp) - F.unix_timestamp(df.prev_timestamp)) / 60
        )
    )

    # Seleziona le colonne desiderate
    return df.drop("prev_timestamp")

In [ ]:
df = add_minutes_since_last_trans(df)

In [ ]:
df.show(5)

### Count how many transactions are sent to unique accounts and bank in different period of time

In [ ]:
def add_unique_to_bank_account(df):
    df = df.withColumn("timestamp_minutes", F.unix_timestamp("timestamp")/60)


    #Day
    windowSpec = Window.partitionBy("from_account").orderBy("timestamp_minutes")\
                .rangeBetween(-1440, Window.currentRow)
    df = df.withColumn("unique_to_banks_last_day", F.size(F.collect_set("to_bank").over(windowSpec)))
    df = df.withColumn("unique_to_accounts_last_day", F.size(F.collect_set("to_account").over(windowSpec)))


    return df.drop('timestamp_minutes')



In [ ]:
df = add_unique_to_bank_account(df)

### Count how many transactions are sent to unique accounts with same payment format in different period of time

In [ ]:
def add_unique_payment_format(df):
    df = df.withColumn("timestamp_minutes", F.unix_timestamp("timestamp")/60)
    
    #Day
    windowSpec = Window.partitionBy("from_account").orderBy("timestamp_minutes")\
                .rangeBetween(-1440, Window.currentRow)
    df = df.withColumn("unique_payment_formats_last_day", F.size(F.collect_set("payment_format").over(windowSpec)))

    return df.drop('timestamp_minutes')

In [ ]:
df = add_unique_payment_format(df)

### Count how many transactions are sent to unique accounts with same payment currency in different period of time

In [ ]:
def add_unique_payment_currency(df):
    df = df.withColumn("timestamp_minutes", F.unix_timestamp("timestamp")/60)

    #Day
    windowSpec = Window.partitionBy("from_account").orderBy("timestamp_minutes")\
                .rangeBetween(-1440, Window.currentRow)
    df = df.withColumn("unique_payment_currency_last_day", F.size(F.collect_set("payment_currency").over(windowSpec)))

    return df.drop('timestamp_minutes')


In [ ]:
df = add_unique_payment_currency(df)

### Count how many transactions an account execute to the same other account until the transaction itself

In [ ]:
def add_transaction_recurrence(df):
    windowSpec = Window.partitionBy("from_account", "to_account").orderBy("timestamp").rowsBetween(Window.unboundedPreceding, Window.currentRow)
    return df.withColumn("transaction_recurrence", F.count("timestamp").over(windowSpec))

In [ ]:
df = add_transaction_recurrence(df)

In [ ]:
df.show(5)

## Save dataframe in parquet format

In [ ]:
df.coalesce(32).write.parquet('partial_results/df.parquet', mode='overwrite')

## Label encoding

Il label encoding viene fatto su tutte le features non numeriche così da poter successivamente creare la matrice di correlazione e utilizzare le features anche per allenare e testare il modello

In [ ]:
def create_mapping(df, col1, col2=None):
    unique_columns = set(df.select(col1).orderBy(col1).distinct().rdd.flatMap(lambda x: x).collect())
    if col2:
        unique_columns.update(df.select(col2).distinct().rdd.flatMap(lambda x: x).collect())
    return {column: index for index, column in enumerate(unique_columns)}

def label_columns(df, col1, mapping, col2=None):
    column_to_index_udf = udf(lambda column: mapping[column], IntegerType())
    
    df_to_return = df.withColumn(f"{col1}_indexed", column_to_index_udf(col(col1))).drop(col1).withColumnRenamed(f'{col1}_indexed', col1) 
    if col2:
        df_to_return = df_to_return.withColumn(f"{col2}_indexed", column_to_index_udf(col(col2))).drop(col2).withColumnRenamed(f'{col2}_indexed', col2)
    return df_to_return

## Display correlation matrix

A correlation matrix is a table showing the correlation coefficients between variables. Each cell in the table shows the correlation between two variables. The value is in the range of -1 to 1. If two variables have high correlation, it means when one variable changes, the second tends to change in a specific direction. A value close to 1 implies a strong positive correlation: as one variable increases, the other also tends to increase. A value close to -1 implies a strong negative correlation: as one variable increases, the other tends to decrease. A value close to 0 implies a weak or no linear correlation between the variables.

In [ ]:
from sklearn.preprocessing import LabelEncoder
def encode_columns(df, col1, col2=None):
    le = LabelEncoder()
    if col2:
        le.fit(list(set(df[col1]).union(set(df[col2]))))
        df[col1] = le.transform(df[col1])
        df[col2] = le.transform(df[col2])
    else:
        df[col1] = le.fit_transform(df[col1])
    return df

In [ ]:
df = pd.read_parquet('partial_results/df.parquet')

df = encode_columns(df, 'payment_format')
df = encode_columns(df, 'from_account', 'to_account')
df = encode_columns(df, 'receiving_currency', 'payment_currency')

df = df.drop('timestamp', axis=1)

laundering_corr_matrix = df.corr()

fig, ax = plt.subplots(figsize=(22, 8))
sns.heatmap(laundering_corr_matrix, cmap='inferno', annot=False, ax=ax)
ax.set_title('Correlation matrix')
plt.show()

La matrice di correlazione mostra come non ci siano forti correlazioni tra le etichette e le features prese in esame. Viceversa si può notare come ci siano features strettamente correlate tra di loro che potrebbero causare ridondanza nei dati oltre che aumentare il numero di features rendendo il modello più complesso e causare overfitting. Le forti correlazioni vengono però risolte in quanto viene utilizzato un modello di apprendimento random forest che va a campionare e selezionare diverse features per ogni albero.




# Data Organization and Pre-processing

In questa sezione andremo a dividere il dataset per effettuare il training e il test dei modelli. La divisione scelta è 60% per il Train set, il 20% per il validation set (Hyperparameter tuning) e il 20% per il Test set. La divisione è stata suggerita dalla descrizione presente sul sito Kaggle riguardante questo dataset.

Una volta diviso il dataset verrà eseguito il preprocessing dei dati, dunque verranno tolte le features che non sono utili ai fini della predizione e infine verranno salvati i diversi set di dati per essere utilizzati.

## Data splitting

Come detto precedentemente, i dati vengono divisi in 60% train, 20% validation e 20% test

Prima di effettuare la divisione i dati vengono ordinati per timestamp, per poi suddividerli in maniera sequenziale. Di norma questa procedura non andrebbe fatta, ma viene effettuata in quanto la logica delle transazioni si basa sul tempo e ci sono molte features collegate ad esso. Se i dati fossero presi in maniera randomica, le features temporali assumerebbero molta meno importanza.


In [ ]:
df = spark.read.parquet('partial_results/df.parquet')
ordered_df = df.orderBy("timestamp")

mapping_format = create_mapping(df, 'payment_format')
mapping_currency = create_mapping(df, 'payment_currency', 'receiving_currency')

ordered_df.cache()
# Calculate row counts for splits
total_rows = ordered_df.count()
train_rows, validation_rows = int(total_rows * 0.6), int(total_rows * 0.2)
test_rows = total_rows - train_rows - validation_rows

# Add a dummy partition and assign row numbers based on ordered timestamps
w = Window.partitionBy(lit(1)).orderBy("timestamp")
ordered_df = ordered_df.withColumn("row_number", F.row_number().over(w))

# Split and repartition the DataFrame into train, validation, and test sets based on row numbers
train_df = ordered_df.filter(col("row_number") <= train_rows).drop("row_number", "dummy_partition").repartition(32)
validation_df = ordered_df.filter(col("row_number").between(train_rows + 1, train_rows + validation_rows)).drop("row_number", "dummy_partition").repartition(32)
test_df = ordered_df.filter(col("row_number") > train_rows + validation_rows).drop("row_number", "dummy_partition").repartition(32)

## Calculate features over splitted data and save

In questa sezione vado ad effettuare il pre-processing sui dati. Vengono calcolate dunque tutte le features necessarie che sono state utilizzate per l'analisi e vengono rimosse le features che non si reputa possano essere importanti.

In questo caso le features rimosse sono la banca di provenienza ed arrivo in quanto ci si aspetta che siano valori di poco conto in un sistema in cui magari si allena il modello su determinati tipi di banche e magari lo si usa per predirre dati in cui non compaiono le stesse. Il ragionamento è il medesimo per l'account di provenienza e arrivo. E' stato deciso inoltre di andare a togliere gli amount in quanto già dalle analisi si dimostravano poco utili e molto randomici.

Index e timestamp vengono tolti in quanto inutili per l'addestramento e l'utilizzo del modello.

In [ ]:
def remove_unnecessary_features(df):
    df = df.drop('from_bank')
    df = df.drop('to_bank')
    df = df.drop('from_account')
    df = df.drop('to_account')
    df = df.drop('amount_received')
    df = df.drop('amount_paid')
    df = df.drop('index')
    df = df.drop('timestamp')
    return df

I dati vengono salvati in formato parquet così da poterli riutilizzare senza ricalcolare ogni volta tutte le features.

In [ ]:
def preprocess_df(df, name):
    df = label_columns(df, 'receiving_currency', mapping_currency, 'payment_currency')
    df = label_columns(df,'payment_format', mapping_format)

    df = add_same_account(df)
    df = add_same_bank(df)
    df = add_same_currency(df)
    df = add_trans_received(df)
    df = add_trans_send(df)
    df = add_minutes_since_last_trans(df)
    df = add_unique_to_bank_account(df)
    df = add_unique_payment_format(df)
    df = add_unique_payment_currency(df)
    df = add_transaction_recurrence(df)
    df = remove_unnecessary_features(df)
    df.coalesce(32).write.parquet(f'partial_results/df.{name}', mode='overwrite')
    return df

In [ ]:
train_df = preprocess_df(train_df, 'train')
test_df = preprocess_df(test_df, 'test')
validation_df = preprocess_df(validation_df, 'val')
ordered_df.unpersist()

## Feature selection

I dataset precedentemente salvati contengono un elevato numero di features, ma non per forza tutte le feature sono utili per un buon modello predittivo. Per questo motivo ho deciso di utilizzare una tecnica di feature selection denominata Boruta.

Boruta è un algoritmo di selezione delle features per modelli basati su random forest. Esso identifica e conserva le caratteristiche più importanti per la previsione, eliminando quelle irrilevanti o ridondanti. Boruta agisce confrontando l'importanza delle caratteristiche originali con quella di caratteristiche casuali "ombra" (shadow features). Se una caratteristica originale è meno importante di una caratteristica ombra, viene considerata irrilevante e rimossa.

In [ ]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

train_pd = pd.read_parquet('./partial_results/df.train')

X_train = train_pd.drop('is_laundering', axis=1)
y_train = train_pd['is_laundering']

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample', n_estimators=30)

# Definisci Boruta
boruta_selector = BorutaPy(rf, n_estimators='auto', random_state=0)

# Addestra Boruta
boruta_selector.fit(np.array(X_train.values), np.array(y_train.values.ravel()))

# Seleziona le caratteristiche rilevanti
X_train_selected = boruta_selector.transform(X_train.values)

In [ ]:
selected_features = X_train.columns[boruta_selector.support_]
removed_features = X_train.columns[[True if x == False else False for x in boruta_selector.support_]]
print("Selected Features:", selected_features)
print("Removed Features:", removed_features)

Boruta ha selezionato quattro features che reputa non utili, dunque andrò a toglierle dai set di dati. Da notare come tra le features non selezionate vi è 'same_currency', che già dalle analisi preliminari risultava poco significativa.

In [ ]:
def remove_boruta_selected_features(df, name):
    df = df.drop('same_currency')
    df = df.drop('week')
    df = df.drop('receiving_currency')
    df = df.drop('payment_currency')

    df.coalesce(32).write.parquet(f'partial_results/df.{name}', mode='overwrite')

In [ ]:
remove_boruta_selected_features(train_df, 'train')
remove_boruta_selected_features(test_df, 'test')

# Decision Tree Classifier

A questo punto è possibile andare ad implementare l'albero decisionale from scratch. Un Decision Tree Classifier è un algoritmo di apprendimento supervisionato che utilizza una struttura ad albero per prendere decisioni. Ogni nodo dell'albero rappresenta una domanda o una condizione sui dati, mentre ogni ramo rappresenta un possibile risultato di quella domanda. Le foglie dell'albero rappresentano le classificazioni finali dei dati. L'algoritmo apprende come suddividere i dati basandosi sulle caratteristiche (features) per produrre le decisioni/classificazioni più accurate possibili.

 L'albero verrà testato utilizzando inizialmente dei dati sample presi dalla librearia di scikit-learn. Per questo scopo ho deciso di utilizzare il dataset Breast Tumor in quanto presenta una classificazione binaria e un numero elevato di features, così da poter testare l'efficacia dell'albero. Verrà poi messo a confronto con il decision tree classifier disponibile direttamente nella libreria di scikit-learn.

## Implementazione dell'albero from scratch



In [ ]:
class DTC():
    def __init__(self, random_state = None, max_depth = None, min_sample_split = 2, criterion = "gini", min_info_gain = 0, max_features = None, max_thresholds = None, class_weights = {}, verbose=False):
        """
        Initializes the Decision Tree Classifier.

        Parameters:
        - random_state: int or None, optional (default=None)
            Seed for the random number generator.
        - max_depth: int or None, optional (default=None)
            The maximum depth of the decision tree. If None, the tree is grown until all leaves are pure or until all leaves contain less than min_sample_split samples.
        - min_sample_split: int, optional (default=2)
            The minimum number of samples required to split an internal node.
        - criterion: str, optional (default="gini")
            The function to measure the quality of a split. Supported criteria are "gini" for the Gini impurity, "entropy" for the information gain, and "shannon" for the Shannon entropy.
        - min_info_gain: float, optional (default=0)
            The minimum information gain required to split an internal node.
        - max_features: int, float, "sqrt", "log2", or None, optional (default=None)
            The number of features to consider when looking for the best split. If int, then consider max_features features at each split. If float, then max_features is a fraction and int(max_features * n_features) features are considered. If "sqrt", then max_features=sqrt(n_features). If "log2", then max_features=log2(n_features). If None, then all features are considered.
        - max_thresholds: int or None, optional (default=None)
            The maximum number of thresholds to consider for each feature when looking for the best split. If None, all unique feature values are considered as potential thresholds.
        - class_weights: dict, optional (default={})
            Weights associated with classes. If provided, the class probabilities are multiplied by the corresponding weight.

        Returns:
        None
        """
        
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.criterion = criterion
        self.min_info_gain = min_info_gain
        self.max_features = max_features
        self.max_thresholds = max_thresholds
        self.class_weights = class_weights
        self.tree = None
        self.random_state = random_state
        np.random.seed(self.random_state)
        self.node_counter = 0
        self.verbose = verbose
        

    def fit(self, X, y):
        """
        Fits the decision tree classifier to the training data.

        Parameters:
        - X: array-like of shape (n_samples, n_features)
            The training input samples.
        - y: array-like of shape (n_samples,)
            The target values.

        Returns:
        None
        """
        self.node_counter = 0
        if isinstance(X, pd.DataFrame):
            X = np.array(X)
        if isinstance(y, pd.DataFrame):
            y = np.array(y)

        self.tree = self.__create_tree(X, y)
       
    def __calculate_split_entropy(self, y):
        """
        Calculates the split entropy for a given target variable.

        Parameters:
        - y: array-like of shape (n_samples,)
            The target values.

        Returns:
        split_criterion: float
            The split criterion value.
        """

        unique_values, value_counts = np.unique(y, return_counts=True)
        class_probabilities = value_counts / len(y)

        if len(self.class_weights) > 0 and len(unique_values) > 0:
            class_probabilities *= np.array([self.class_weights.get(value, 1.0) for value in unique_values])

        split_criteria = {
            'shannon': lambda probs: -np.sum(probs * np.log2(probs)),
            'entropy': lambda probs: -np.sum(probs * np.log2(probs + 1e-10)),
            'gini': lambda probs:  1 - np.sum(probs ** 2)
        }

        split_criterion_function = split_criteria.get(self.criterion, split_criteria['gini'])
        split_criterion = split_criterion_function(class_probabilities)

        return split_criterion


    def __calculate_info_gain(self, X, y, feature, threshold):
        """
        Calculates the information gain for a given feature and threshold.

        Parameters:
        - X: array-like of shape (n_samples, n_features)
            The input samples.
        - y: array-like of shape (n_samples,)
            The target values.
        - feature: int
            The index of the feature.
        - threshold: float
            The threshold value.

        Returns:
        info_gain: float
            The information gain.
        """

        left_indeces = X[:, feature] <= threshold
        right_indeces = X[:, feature] > threshold

        left_labels = y[left_indeces]
        right_labels = y[right_indeces]

        left_side_entropy = self.__calculate_split_entropy(left_labels)
        right_side_entropy = self.__calculate_split_entropy(right_labels)

        
        weighted_left_side_entropy = (len(left_labels) / len(y)) * left_side_entropy
        weighted_right_side_entropy = (len(right_labels) / len(y)) * right_side_entropy

        parent_entropy = self.__calculate_split_entropy(y)

        info_gain = parent_entropy - (weighted_left_side_entropy + weighted_right_side_entropy)

        return info_gain


    def __get_features(self, n_features):
        """
        Returns the indices of the features to consider for splitting.

        Parameters:
        - n_features: int
            The total number of features.

        Returns:
        columns_id: list
            The indices of the features to consider.
        """

        np.random.seed(self.random_state + self.node_counter if self.random_state is not None else None)
        if self.max_features is not None:
            if self.max_features == "sqrt":
                columns_id = np.random.choice(range(n_features), int(math.sqrt(n_features)), replace=False)
            elif self.max_features == "log2":
                columns_id = np.random.choice(range(n_features), int(math.log2(n_features)), replace=False)
            elif isinstance(self.max_features, int):
                if self.max_features > n_features:
                    raise ValueError("Max features > number of features")
                elif self.max_features <= 0:
                    raise ValueError("Max features must be > 0")
                columns_id = np.random.choice(range(n_features), self.max_features, replace=False)
            elif isinstance(self.max_features, float):
                if self.max_features > 1:
                    raise ValueError("Max features > number of features")
                elif self.max_features <= 0:
                    raise ValueError("Max features must be > 0")
                columns_id = np.random.choice(range(n_features), int(n_features * self.max_features), replace=False)
        else:
            columns_id = list(range(n_features))    

        return columns_id
    

    def __get_thresholds(self, X, feature):
        """
        Returns the thresholds to consider for a given feature.

        Parameters:
        - X: array-like of shape (n_samples, n_features)
            The input samples.
        - feature: int
            The index of the feature.

        Returns:
        thresholds: array-like
            The thresholds to consider.
        """
        np.random.seed(self.random_state + self.node_counter if self.random_state is not None else None)
        if self.max_thresholds is not None:
            if self.max_thresholds <= 0:
                raise ValueError("max_thresholds must be > 0")
            thresholds = np.percentile(X[:, feature], np.linspace(0, 100, self.max_thresholds))
        else:
            unique_vals = np.unique(X[:, feature])
            thresholds = (unique_vals[1:] + unique_vals[:-1]) / 2
        return thresholds



    def __calculate_best_split(self, X, y):
        """
        Calculates the best split for the given input samples and target values.

        Parameters:
        - X: array-like of shape (n_samples, n_features)
            The input samples.
        - y: array-like of shape (n_samples,)
            The target values.

        Returns:
        best_feature: int
            The index of the best feature to split on.
        best_threshold: float
            The best threshold value.
        best_info_gain: float
            The information gain of the best split.
        """

        best_threshold = None
        best_info_gain = -np.inf
        best_feature = None
        self.node_counter += 1
        features = self.__get_features(X.shape[1])

        for feature in features:
            threholds = self.__get_thresholds(X, feature)
               
            for threshold in threholds:
                info_gain = self.__calculate_info_gain(X, y, feature, threshold)
                if best_info_gain < info_gain:
                    best_threshold = threshold
                    best_feature = feature
                    best_info_gain = info_gain

        return best_feature, best_threshold, best_info_gain


    def __create_tree(self, X, y, depth = 0):
        """
        Recursively creates the decision tree.

        Parameters:
        - X: array-like of shape (n_samples, n_features)
            The input samples.
        - y: array-like of shape (n_samples,)
            The target values.
        - depth: int, optional (default=0)
            The current depth of the tree.

        Returns:
        tree: dict
            The decision tree.
        """
        samples = X.shape[0]
        
        if samples < self.min_sample_split or (self.max_depth != None and depth >= self.max_depth):
            return self.__create_leaf_node(y)

        best_feature, best_threshold, best_info_gain = self.__calculate_best_split(X, y)

        if(best_info_gain <= self.min_info_gain):
            return self.__create_leaf_node(y)
        
        left_indices = X[:, best_feature] <= best_threshold
        right_indices = X[:, best_feature] > best_threshold

        left_child = self.__create_tree(X[left_indices], y[left_indices], depth + 1)
        right_child = self.__create_tree(X[right_indices], y[right_indices], depth + 1)

        if 'label' in left_child and 'label' in right_child:
            if left_child['label'] == right_child['label']:
                return {
                    'label': left_child['label'],
                    'samples': len(y)
                }


        return {
            'splitting_threshold': best_threshold,
            'splitting_feature': best_feature,
            'info_gain': best_info_gain,
            'left_child': left_child,
            'right_child': right_child
        }
    


    def __create_leaf_node(self, y):
        """
        Creates a leaf node for the decision tree.

        Parameters:
        - y: array-like of shape (n_samples,)
            The target values.

        Returns:
        leaf_node: dict
            The leaf node.
        """
        majority_class = Counter(y).most_common(1)[0][0]
        return {
            'label': majority_class,
            'samples': len(y)
        }
    

    def predict(self, X):
        """
        Predicts the class labels for the input samples.

        Parameters:
        - X: array-like of shape (n_samples, n_features)
            The input samples.

        Returns:
        predictions: array-like of shape (n_samples,)
            The predicted class labels.
        """

        if(isinstance(X, pd.DataFrame)):
            X = np.array(X)
        
        predictions = []
        for sample in X:
            prediction = self.__traverse_tree(sample, self.tree)
            predictions.append(prediction)
        return np.array(predictions)

    def __calculate_metrics(self, y_true, y_pred):
        # Inizializza le variabili
        TP = 0
        TN = 0
        FP = 0
        FN = 0
        
        # Conta TP, TN, FP, FN
        for true, pred in zip(y_true, y_pred):
            if true == 1 and pred == 1:
                TP += 1
            elif true == 0 and pred == 0:
                TN += 1
            elif true == 1 and pred == 0:
                FN += 1
            elif true == 0 and pred == 1:
                FP += 1
        
        # Calcola precisione, recall, f1-score e accuracy
        precision = TP / (TP + FP) if (TP + FP) != 0 else 0
        recall = TP / (TP + FN) if (TP + FN) != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
        accuracy = (TP + TN) / (TP + TN + FP + FN)
        
        return precision, recall, f1_score, accuracy

    def score(self, X, y):
        if(isinstance(y, pd.DataFrame)):
            y = np.array(y)


        y_pred = self.predict(X)
        if(self.verbose):
            precision, recall, f1_score, accuracy = self.__calculate_metrics(y, y_pred)
            metrics = {
                'accuracy': accuracy,
                'recall': recall,
                'precision': precision,
                'f1_score': f1_score
            }

            print(metrics)
            print(f"max_depth: {self.max_depth}, min_sample_split: {self.min_sample_split}, criterion: {self.criterion}, max_features: {self.max_features}, max_thresholds: {self.max_thresholds}")
            
        return np.mean(y_pred == y)
    

    def __traverse_tree(self, sample, node):
        """
        Traverses the decision tree to predict the class label for a given sample.

        Parameters:
        - sample: array-like of shape (n_features,)
            The input sample.
        - node: dict
            The current node of the decision tree.

        Returns:
        label: int
            The predicted class label.
        """

        if 'label' in node:
            return node['label']
        else:
            if sample[node['splitting_feature']] <= node['splitting_threshold']:
                return self.__traverse_tree(sample, node['left_child'])
            else:
                return self.__traverse_tree(sample, node['right_child'])
            
    def __recursive_print(self, node, indent=""):
        """Recursively print the decision tree.

        Parameters
        ----------
        node
            The current node to be printed.
        indent : str, default=""
            The indentation string for formatting the tree.
        """
        if 'label' in node:
            print("{}leaf - label: {} ".format(indent, node['label']))
            return

        print("{}id:{} - threshold: {}".format(indent,
              node['splitting_feature'], node['splitting_threshold']))

        self.__recursive_print(node['left_child'], "{}   ".format(indent))
        self.__recursive_print(node['right_child'], "{}   ".format(indent))

    def print_tree(self):
        """Display the decision tree structure."""
        self.__recursive_print(self.tree)

    def get_params(self, deep=True):
        """Get parameters for this estimator.

        Parameters:
        - deep: bool, default=True
            If True, will return the parameters for this estimator and contained subobjects that are estimators.

        Returns:
        params: mapping of string to any
            Parameter names mapped to their values.
        """
        return {
            "random_state": self.random_state,
            "max_depth": self.max_depth,
            "min_sample_split": self.min_sample_split,
            "criterion": self.criterion,
            "min_info_gain": self.min_info_gain,
            "max_features": self.max_features,
            "max_thresholds": self.max_thresholds,
            "class_weights": self.class_weights,
            "verbose": self.verbose
        }

    def set_params(self, **params):
        """Set the parameters of this estimator.

        Parameters:
        - **params: dict
            Estimator parameters.

        Returns:
        self: estimator instance
            Estimator instance.
        """
        for key, value in params.items():
            setattr(self, key, value)
        return self

    

## Test with scikit-learn library

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

data = load_breast_cancer(as_frame=False)

X_train, X_test, y_train, y_test = train_test_split(np.array(data['data']), np.array(data['target']), test_size=0.2, random_state=42)

In [ ]:
clf = DecisionTreeClassifier(random_state=0)
dtc = DTC(random_state=0)

In [ ]:
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
dtc.fit(X_train, y_train)
dtc.score(X_test, y_test)

## Hyperparameter Tuning

Dopo aver implementato l'albero, prima di poter andare a testarlo in maniera definitiva sui dati, vado ad effettuare un tuning degli iperparametri così da massimizzare le prestazioni del modello. Il tuning viene fatto utilizzando la libreria Hyperopt la quale va ad eseguire il train e calcolare lo score sul validation set con diverse combinazioni di iperparametri.

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, SparkTrials

train_pd = pd.read_parquet('./partial_results/df.train')
val_pd = pd.read_parquet('./partial_results/df.val')

X_train = np.array(train_pd.drop('is_laundering', axis=1))
y_train = np.array(train_pd['is_laundering'])

X_val = np.array(val_pd.drop('is_laundering', axis=1))
y_val = np.array(val_pd['is_laundering'])


Il tuning degli iperparametri verrà fatto andando ad aggiungere il class_weighting. Questo perché in questo modo si creare bilanciamento nelle classi quando viene calcolata l'information gain

In [ ]:
class_weights = {0: 1, 1: 1/(np.sum(y_train)/len(y_train))}

In [ ]:
def objective_function(params):
        dtc = DTC(**params, random_state=0, class_weights=class_weights, verbose=True)
        dtc.fit(X_train, y_train)
        return -dtc.score(X_val, y_val)  

Gli iperparametri vengono scelti in base ad intervalli di due valori ciascuno, così da ottenere più combinazioni. Nella scelta degli iperparametri non viene considerato il "None" quando si va a prendere la profondità massima, le max_features e i max_thresholds in quanto comporterebbe un enorme dispendio di tempo calcolare le diverse combinazioni.

In [ ]:
choises = {
    'max_depth': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
    'min_sample_split': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
    'criterion': ['gini', 'entropy', 'shannon'],
    'max_features': ['sqrt', 'log2'],
    'max_thresholds': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
}

space = {
    'max_depth': hp.choice('max_depth', choises['max_depth']),
    'min_sample_split': hp.choice('min_sample_split', choises['min_sample_split']),
    'criterion': hp.choice('criterion', choises['criterion']),
    'max_features': hp.choice('max_features', choises['max_features']),
    'max_thresholds': hp.choice('max_thresholds', choises['max_thresholds'])
}

In [ ]:
trials = Trials()
best = fmin(fn=objective_function, space=space, algo=tpe.suggest, max_evals=300, trials=trials)

Una volta eseguito l'hyperparameter tuning, fatto per 600 iterazioni (1/10 delle possibili totali), i parametri scelti per il miglior score ottenuto, verranno utilizzati come parametri da inserire per inizializzare il decision tree

In [ ]:
for key, value in best.items():
    print(f"{key}: {choises[key][value]}")

## Train and Test DTC

In questa fase viene utilizzata

In [ ]:
test_pd = pd.read_parquet('./partial_results/df.test')

X_test = np.array(test_pd.drop('is_laundering', axis=1))
y_test = np.array(test_pd['is_laundering'])

cls_m = DTC(random_state=0, max_thresholds = 10, criterion='shannon', max_depth=25, min_sample_split=14, max_features='log2', verbose=True)
cls_m.fit(X_res, y_res)
cls_m.score(X_test, y_test)
y_pred = cls_m.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Calcola la matrice di confusione
cm = confusion_matrix(y_test, y_pred)

# Etichette delle classi (sostituisci con i tuoi nomi di classi reali se necessario)
class_names = ['0', '1']

# Crea il grafico della matrice di confusione
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Matrice di Confusione')
plt.colorbar()

tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)

# Aggiungi i valori delle celle nella matrice
thresh = cm.max() / 2.0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.xlabel('Classe Predetta')
plt.ylabel('Classe Reale')
plt.tight_layout()
plt.show()